In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
import json
import numpy as np
from IPython.display import display
from subprocess import check_output
import sys
import traceback
from tqdm.notebook import tqdm, trange
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from paramiko import SSHClient
from scp import SCPClient
from subprocess import call
pd.options.mode.chained_assignment = None
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
dat_2 = pd.read_pickle("two_players_data")
dat_3 = pd.read_pickle("three_players_data")
dat_4 = pd.read_pickle("four_players_data")

In [3]:
techs = {
    "tech1": "1q_1o",
    "tech2": "k_per_terra",
    "tech3": "4pip",
    "tech4": "7vp",
    "tech5": "1o_1pw",
    "tech6": "1k_1c",
    "tech7": "3vp_per_gaia_place",
    "tech8": "4c",
    "tech9": "4pw"
}

adv_techs = {
    "advtech1": "3vp_per_fed_pass",
    "advtech2": "2vp_per_tech_bump",
    "advtech3": "1qic_5c_action",
    "advtech4": "2vp_per_mine",
    "advtech5": "3vp_per_rl_pass",
    "advtech6": "1o_per_sector",
    "advtech7": "1vp_per_terra_pass",
    "advtech8": "2vp_per_gaia",
    "advtech9": "4vp_per_ts",
    "advtech10": "2vp_per_sector",
    "advtech11": "3o_action",
    "advtech12": "5vp_per_fed",
    "advtech13": "3k_action",
    "advtech14": "3vp_per_mine_place",
    "advtech15": "3vp_per_ts_place",
}

feds = {
    "fed1": "12vp",
    "fed2": "qic",
    "fed3": "2pw",
    "fed4": "2o",
    "fed5": "6c",
    "fed6": "2k",
    "gleens": "gleens"
}

round_scorings = {
    "score1": "2vp_per_terra",
    "score2": "2vp_per_research_bump",
    "score3": "2vp_per_mine_place",
    "score4": "5vp_per_fed_place",
    "score5": "4vp_per_ts_place",
    "score6": "4vp_per_gaia_place",
    "score7": "5vp_per_3pip_place",
    "score8": "3vp_per_ts_place",
    "score9": "3vp_per_gaia_place",
    "score10": "5vp_per_3pip_place",
}

boosters = {

    "booster1": "1k_1o",
    "booster2": "2pwt_1o",
    "booster3": "1qic_2c",
    "booster4": "2c_terra",
    "booster5": "2pw_nav",
    "booster6": "1o_1vp_per_mine",
    "booster7": "1o_2vp_per_ts",
    "booster8": "1k_3vp_per_rl",
    "booster9": "4pw_4vp_per_3pip",
    "booster10": "4c_1vp_per_gaia",
}

bad_buildings = {"colony", 
                 "colonyShip", 
                 "tradeShip", 
                 "constructionShip", 
                 "researchShip",
                 "scout",
                 "frigate",
                 "battleShip",
                 "customsPost",
                 "tradePost"
                }

factions = {
    "terrans",
    "lantids",
    "hadsch-hallas",
    "ivits",
    "baltaks",
    "geodens",
    "xenos",
    "gleens",
    "ambas",
    "taklons",
    "bescods",
    "firaks",
    "itars",
    "nevlas"
}

final_scorings = {
    "planetType",
    "structure", 
    "sector", 
    "satellite", 
    "gaia",
    "structureFed"
}

scoring_types = [
    # 'init', 
    # 'bid', 
    'charge',
    'tech_basic', 
    'gleens',
    'round', 
    'booster',
    'qic', 
    'tech_score',
    'gaia', 
    'adv', 
    'fed', 
    'final',
    'resource'
]

In [4]:
def make_corr_plots(dat_x, num_players):
    
    
    # clean dat_x
    clean_pos = []
    for pos in range(num_players):
        # sum round scorings
        dat_x['pos_' + str(pos + 1) + '_score_round'] = 0
        for i in range(6):
            dat_x['pos_' + str(pos + 1) + '_score_round'] += dat_x['pos_' + str(pos + 1) + '_score_round' + str(i + 1)] 
        # sum final scorings
        dat_x['pos_' + str(pos + 1) + '_score_final'] = dat_x['pos_' + str(pos + 1) + '_score_final1'] + dat_x['pos_' + str(pos + 1) + '_score_final2']
        
        # add charge and bid to the total
        dat_x['pos_' + str(pos + 1) + '_score'] += -1 * dat_x['pos_' + str(pos + 1) + '_score_charge']
        # ignore bid bc of position comparison
        # dat_x['pos_' + str(pos + 1) + '_score'] += -1 * dat_x['pos_' + str(pos + 1) + '_score_bid']
        
        # dat = dat[dat['pos_' + str(pos + 1) + '_elo'] > 200]
        clean_pos.append(dat_x)
    dat_clean = pd.concat(clean_pos, ignore_index=True)
    
    # make colors
    c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(120, 0, num_players)]
    
    # make figures
    for faction in factions:
        factions_pos = []
        for pos in range(num_players):
            factions_pos.append(dat_clean[dat_clean["pos_" + str(pos + 1) +"_faction"] == faction])
        
        fig = go.Figure()
        
        for pos in reversed(range(num_players)):
            df = factions_pos[pos]
            df = df[df['pos_' + str(pos + 1) + '_dropped'] == False]
            df = df[df['pos_' + str(pos + 1) + '_score'] > 100]
            x = np.array([])
            y = np.array([])
            for scoring in scoring_types:
                if scoring == 'tech_score':
                    more_y = df['pos_' + str(pos + 1) + '_' + scoring] / df['pos_' + str(pos + 1) + '_score']
                elif scoring == 'charge':
                    more_y = -1 * df['pos_' + str(pos + 1) + '_score_' + scoring] / df['pos_' + str(pos + 1) + '_score']
                else:
                    more_y = df['pos_' + str(pos + 1) + '_score_' + scoring] / df['pos_' + str(pos + 1) + '_score']
                more_y = more_y.values * 100
                more_y = np.round(more_y, 2)
                
                more_x = [scoring] * len(more_y)
                
                x = np.append(x, np.array(more_x))
                y = np.append(y, more_y)
            
            fig.add_trace(go.Box(
                y=y,
                x=x,
                name='rank ' + str(pos + 1),
                boxpoints='outliers',
                boxmean=True,
                marker_color=c[pos]
            ))
                
        fig.update_layout(
            title='Point source breakdown for points made : ' + faction,
            yaxis_title='percent of overall points made',
            xaxis_title='scoring type',
            boxmode='group'
        )
        fig.show()
        

In [ ]:
make_corr_plots(dat_4[[
    
]], 4)

In [ ]:
pos_dat = dat_2[dat_2['pos_1_faction'] == 'ivits']
y = pos_dat['pos_1_score_adv'] / pos_dat['pos_1_score']
y = y.values * 100
y = np.round(y, 2)
y